In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import pandas as pd

In [ ]:

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
])


In [ ]:

# Load the training and testing datasets
train_data = datasets.ImageFolder('path_to_train_data', transform=transform)
test_data = datasets.ImageFolder('path_to_test_data', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
# Load the pre-trained ResNet18 model and modify the final layer for binary classification
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust the final layer for binary classification


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

In [ ]:
# Testing the model and preparing the results for the excel sheet
model.eval()
results = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(images)):
            image_name = test_data.imgs[i][0].split('/')[-1]  # Extract image name from path
            emergency_or_not = 'Yes' if predicted[i].item() == 1 else 'No'
            results.append([image_name, emergency_or_not])

In [ ]:
# Create a DataFrame and save to an excel sheet
df = pd.DataFrame(results, columns=['image_names', 'emergency_or_not'])
df.to_excel('emergency_vehicle_results.xlsx', index=False)

print("Results have been saved to emergency_vehicle_results.xlsx")